In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
from youdied.src.inspiration_message import messages

In [3]:
# Define and create a folder in the user's folder
storage_folder = Path().home() / '.youdied'
# simple death count .txt file folder
death_count_folder = storage_folder / 'deathCount'

# create if not exists
storage_folder.mkdir(parents=True, exist_ok=True)
death_count_folder.mkdir(parents=True, exist_ok=True)

# define files to store statistics
session_file = storage_folder / 'sessions.csv'  # Store character names (and session durations?)
death_file = storage_folder / 'deaths.csv'  # Long format of character deaths
message_file = storage_folder/ 'messages.csv'

In [278]:
session_columns = ['Character', 'Started']
death_columns = ['Character', 'TimeOfDeath']
message_columns = ['Message', 'Used']

In [279]:
def initialize_storage():
    # create files if not existing
    if not session_file.exists():
        session_df = pd.DataFrame(columns=session_columns)
        session_df.to_csv(session_file)
    if not death_file.exists():
        death_df = pd.DataFrame(columns=death_columns)
        death_df.to_csv(death_file)
    if not message_file.exists():
        message_df = pd.DataFrame(((p, 0) for p in messages), columns=message_columns)
        message_df.to_csv(message_file, encoding='utf8')
        
    session_df = pd.read_csv(session_file, index_col=0)
    death_df = pd.read_csv(death_file, index_col=0)
    message_df = pd.read_csv(message_file, index_col=0)
    return session_df, death_df, message_df

In [280]:
# random names
characters = ['Tarnished finger', 'Always Feet 69', 'Elden Lard', 'Cavelady Dida']

def gen_dummy_session_data():
    for i in range(1, 21):
        character = np.random.choice(characters)
        start = pd.Timestamp(f'2022-04-{i:02d}T12:00:00')
        yield (character, start)
        
        
def gen_dummy_death_data():
    for i in range(1, 21):
        character = np.random.choice(characters)
        time_of_death = pd.Timestamp(f'2022-04-{i:02d}T12:{np.random.randint(1, 60):02d}:{np.random.randint(1, 60):02d}')
        yield (character, time_of_death)
    

In [281]:
session_df = pd.DataFrame(gen_dummy_session_data(), columns=session_columns)
session_df

,Character,Started
0,Cavelady Dida,2022-04-01 12:00:00
1,Cavelady Dida,2022-04-02 12:00:00
2,Cavelady Dida,2022-04-03 12:00:00
3,Elden Lard,2022-04-04 12:00:00
4,Always Feet 69,2022-04-05 12:00:00
5,Cavelady Dida,2022-04-06 12:00:00
6,Elden Lard,2022-04-07 12:00:00
7,Elden Lard,2022-04-08 12:00:00
8,Cavelady Dida,2022-04-09 12:00:00
9,Cavelady Dida,2022-04-10 12:00:00


In [8]:
death_df = pd.DataFrame(gen_dummy_death_data(), columns=death_columns)
death_df

,Character,TimeOfDeath
0,Tarnished finger,2022-04-01 12:25:06
1,Elden Lard,2022-04-02 12:21:14
2,Cavelady Dida,2022-04-03 12:48:50
3,Cavelady Dida,2022-04-04 12:09:25
4,Tarnished finger,2022-04-05 12:37:51
5,Cavelady Dida,2022-04-06 12:56:43
6,Tarnished finger,2022-04-07 12:32:56
7,Elden Lard,2022-04-08 12:29:50
8,Elden Lard,2022-04-09 12:46:45
9,Always Feet 69,2022-04-10 12:37:21


In [9]:
session_df.to_csv(session_file)
death_df.to_csv(death_file)

In [10]:
tfd = death_df[death_df.Character == 'Tarnished finger']
tfd

,Character,TimeOfDeath
0,Tarnished finger,2022-04-01 12:25:06
4,Tarnished finger,2022-04-05 12:37:51
6,Tarnished finger,2022-04-07 12:32:56
14,Tarnished finger,2022-04-15 12:41:52


In [11]:
tfd.shape[0]

4

In [283]:
session_df, death_df, message_df = initialize_storage()

def write_death_count(character):
    # Write the updated death count to a file in the death_count_folder
    #  with the character name as file name
    n_deaths = death_df[death_df.Character == character].shape[0]
    death_count_file = death_count_folder / f'{character}.txt'
    death_count_file.touch(exist_ok=True)
    death_count_file.write_text(str(n_deaths), encoding='utf8')

def add_death(character, time_of_death):
    new_index = death_df.shape[0]
    # add new row
    death_df.loc[new_index]= (character, time_of_death)
    # save file
    death_df.to_csv(death_file, encoding='utf8')
    # update death count file
    write_death_count(character)
    
def add_session(character):
    new_index = session_df.shape[0]
    session_df.loc[new_index] = (character, pd.Timestamp('now'))
    session_df.to_csv(session_file, encoding='utf8')
    
def create_inspiration_message():
    # create candidates from the least used messages
    candidates = message_df[message_df.Used == message_df.Used.min()]
    # pick a random candidate row
    new_message_ix = np.random.choice(candidates.index)
    # update use count and save to file
    message_df.loc[new_message_ix, 'Used'] += 1
    message_df.to_csv(message_file)
    # get message string
    message = message_df.loc[new_message_ix].Message
    return message

In [313]:
create_inspiration_message()

'A man who cannot tolerate small misfortunes can never accomplish great things.'

In [274]:
message_df.head()

,Message,Used
0,"He who asks is a fool for five minutes, but he...",1
1,Give a man a fish and you feed him for a day. ...,1
2,Do not desire to hurt others in the depths of ...,2
3,"When reading, don’t let a single word escape y...",1
4,Blame yourself as you blame others; forgive ot...,1


In [273]:
# create candidates from the least used messages
candidates = message_df[message_df.Used == message_df.Used.min()]
# pick a random candidate row
new_message_ix = np.random.choice(candidates.index)
# update use count
message_df.loc[new_message_ix, 'Used'] += 1
# get message string
message = message_df.loc[new_message_ix].Message

candidates.shape[0], message

(236,
 'There is a vitality, a life force, an energy, a quickening that is translated through you into action, and because there is only one of you in all time, this expression is unique. And if you block it, it will never exist through any other medium and will be lost.')

In [16]:
add_death('Bob The monster', pd.Timestamp('now'))
death_df.tail()

,Character,TimeOfDeath
16,Elden Lard,2022-04-17 12:21:17
17,Elden Lard,2022-04-18 12:35:10
18,Elden Lard,2022-04-19 12:19:58
19,Always Feet 69,2022-04-20 12:40:14
20,Bob The monster,2022-04-19 02:14:21.805095


In [17]:
(death_count_folder / f'Bob The monster.txt').read_text()

'1'

In [332]:
# get previous (last used character)
'' or session_df.iloc[0:1].iloc[-1].Character

'Tarnished finger'

In [364]:
print(session_df[-1:].to_string(index=False, col_space=20))

           Character              Started                Ended
      Always Feet 69  2022-04-20 12:00:00  2022-04-20 14:00:00


In [337]:
session_df.Character.unique().tolist()

['Tarnished finger', 'Always Feet 69', 'Cavelady Dida', 'Elden Lard']

In [369]:
uc = session_df.Character.unique().tolist()

uc_data = [(session_df[session_df.Character == c].Started.max(), c, death_df[death_df.Character == c].shape[0]) for c in uc]

uc_df = pd.DataFrame(uc_data, columns=['Started last', 'Character', 'Death count']).sort_values(by='Started last', ascending=False)

print(uc_df.to_string(index=False, col_space=20))


        Started last            Character          Death count
 2022-04-20 12:00:00       Always Feet 69                    4
 2022-04-17 12:00:00        Cavelady Dida                    4
 2022-04-12 12:00:00           Elden Lard                    8
 2022-04-09 12:00:00     Tarnished finger                    4


In [370]:
l = 'bob the builder'.split(' ')

In [372]:
' '.join(l)

'bob the builder'